# MNIST Rakam Tanıma: Yapay Sinir Ağı Implementasyonu ve Analizi

Bu belge, MNIST veri seti üzerinde sıfırdan bir yapay sinir ağı (Neural Network) implementasyonunu, değerlendirmesini ve görselleştirmesini içermektedir. Tüm süreç adım adım açıklanmıştır. Sinir ağı çalışma mantığı numpy ile işin matematiğine değinerek eğitilmiştir.

## 1. Gerekli Kütüphanelerin İçe Aktarılması

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

Bu kod bloğunda, projemiz için gerekli olan kütüphaneleri içe aktarıyoruz:
- `numpy`: Matematiksel işlemler ve dizi (array) manipülasyonu için
- `tensorflow` ve ilgili modüller: MNIST veri setini yüklemek ve kategori dönüşümleri için
- `sklearn` modülleri: Veri bölme ve performans metriklerini hesaplamak için
- `matplotlib` ve `seaborn`: Veri görselleştirme için

## 2. Aktivasyon Fonksiyonlarının Tanımlanması

In [2]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

Yapay sinir ağımızda kullanacağımız aktivasyon fonksiyonlarını tanımlıyoruz:
- `relu`: Gizli katman için Rectified Linear Unit aktivasyon fonksiyonu
- `relu_derivative`: Geri yayılım (backpropagation) için ReLU'nun türevi
- `softmax`: Çıkış katmanı için olasılık dağılımı oluşturan aktivasyon fonksiyonu

ReLU fonksiyonu, giriş değeri 0'dan büyükse girişi olduğu gibi döndürür, değilse 0 döndürür. Softmax fonksiyonu ise tüm çıkışların toplamının 1 olacağı şekilde bir olasılık dağılımı oluşturur.

## 3. Performans Metriklerini Hesaplama Fonksiyonu

In [3]:
def calculate_metrics(y_true, y_pred):
    y_true_class = np.argmax(y_true, axis=1)
    y_pred_class = np.argmax(y_pred, axis=1)

    # Sınıf bazında precision ve recall
    precision = precision_score(y_true_class, y_pred_class, average='macro')
    recall = recall_score(y_true_class, y_pred_class, average='macro')
    f1 = f1_score(y_true_class, y_pred_class, average='macro')

    return precision, recall, f1

Bu fonksiyon, modelin performansını değerlendirmek için üç önemli metriği hesaplar:
- `precision`: Modelin pozitif olarak tahmin ettiği örnekler içinde gerçekten pozitif olanların oranı
- `recall`: Gerçekte pozitif olan örnekler içinde modelin pozitif olarak tahmin ettiği örneklerin oranı
- `f1`: Precision ve recall'un harmonik ortalaması

"macro" ortalama, her sınıfın eşit ağırlıkta hesaba katıldığı anlamına gelir.

## 4. Veri Setinin Yüklenmesi ve Ön İşleme

In [4]:
# MNIST Veri Setini Yükleme
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizasyon ve Dönüştürme
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Eğitim Setini Validation İçin Bölme (%80 eğitim, %20 validasyon)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Bu kod bloğunda:
1. MNIST veri setini TensorFlow aracılığıyla yüklüyoruz
2. Görüntüleri düzleştiriyor (28x28 -> 784) ve 0-1 aralığına normalize ediyoruz
3. Etiketleri one-hot encoding formatına dönüştürüyoruz
4. Eğitim verilerini %80 eğitim, %20 doğrulama olacak şekilde ayırıyoruz

## 5. Model Parametrelerinin Başlatılması

In [5]:
# Model Ağırlıklarını Başlatma
input_size = 784
hidden_size = 128
output_size = 10
learning_rate = 0.01
epochs = 10
batch_size = 32

W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))

# Metrik takibi için listeler
train_losses = []
val_losses = []
val_accuracies = []
val_precisions = []
val_recalls = []
val_f1s = []

Bu kısımda:
1. Modelin hiperparametrelerini tanımlıyoruz (girişler, gizli katman boyutu, çıkışlar, öğrenme oranı)
2. Ağırlıkları (W1, W2) küçük rastgele değerlerle, bias'ları (b1, b2) sıfırla başlatıyoruz
3. Eğitim sırasında metrikleri takip etmek için boş listeler oluşturuyoruz

## 6. Model Eğitimi ve Metrik Hesaplaması

In [6]:
# Eğitim Döngüsü
for epoch in range(epochs):
    epoch_losses = []

    # Mini-batch training
    for i in range(0, x_train.shape[0], batch_size):
        x_batch = x_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        # İleri Yayılım
        Z1 = np.dot(x_batch, W1) + b1
        A1 = relu(Z1)
        Z2 = np.dot(A1, W2) + b2
        A2 = softmax(Z2)

        # Kayıp (Cross-Entropy)
        loss = -np.sum(y_batch * np.log(A2 + 1e-8)) / batch_size
        epoch_losses.append(loss)

        # Geri Yayılım
        dZ2 = A2 - y_batch
        dW2 = np.dot(A1.T, dZ2) / batch_size
        db2 = np.sum(dZ2, axis=0, keepdims=True) / batch_size

        dZ1 = np.dot(dZ2, W2.T) * relu_derivative(Z1)
        dW1 = np.dot(x_batch.T, dZ1) / batch_size
        db1 = np.sum(dZ1, axis=0, keepdims=True) / batch_size

        # Ağırlık Güncelleme
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1

    # Epoch ortalama kaybı
    avg_train_loss = np.mean(epoch_losses)
    train_losses.append(avg_train_loss)

    # Her Epoch Sonunda Validation Kaybı ve Metrikleri Hesapla
    Z1_val = np.dot(x_val, W1) + b1
    A1_val = relu(Z1_val)
    Z2_val = np.dot(A1_val, W2) + b2
    A2_val = softmax(Z2_val)

    val_loss = -np.sum(y_val * np.log(A2_val + 1e-8)) / x_val.shape[0]
    val_losses.append(val_loss)

    val_accuracy = np.mean(np.argmax(A2_val, axis=1) == np.argmax(y_val, axis=1))
    val_accuracies.append(val_accuracy)

    # Precision ve Recall hesaplama
    val_precision, val_recall, val_f1 = calculate_metrics(y_val, A2_val)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1s.append(val_f1)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.4f} - Val Loss: {val_loss:.4f} - Val Accuracy: {val_accuracy:.4f}")
    print(f"Validation Metrics - Precision: {val_precision:.4f} - Recall: {val_recall:.4f} - F1 Score: {val_f1:.4f}\n")


Epoch 1/10 - Loss: 1.4315 - Val Loss: 0.6102 - Val Accuracy: 0.8415
Validation Metrics - Precision: 0.8424 - Recall: 0.8387 - F1 Score: 0.8384

Epoch 2/10 - Loss: 0.4835 - Val Loss: 0.4046 - Val Accuracy: 0.8882
Validation Metrics - Precision: 0.8869 - Recall: 0.8867 - F1 Score: 0.8865

Epoch 3/10 - Loss: 0.3775 - Val Loss: 0.3478 - Val Accuracy: 0.9038
Validation Metrics - Precision: 0.9027 - Recall: 0.9026 - F1 Score: 0.9024

Epoch 4/10 - Loss: 0.3366 - Val Loss: 0.3186 - Val Accuracy: 0.9116
Validation Metrics - Precision: 0.9106 - Recall: 0.9106 - F1 Score: 0.9104

Epoch 5/10 - Loss: 0.3111 - Val Loss: 0.2985 - Val Accuracy: 0.9166
Validation Metrics - Precision: 0.9157 - Recall: 0.9157 - F1 Score: 0.9155

Epoch 6/10 - Loss: 0.2916 - Val Loss: 0.2822 - Val Accuracy: 0.9213
Validation Metrics - Precision: 0.9205 - Recall: 0.9204 - F1 Score: 0.9203

Epoch 7/10 - Loss: 0.2751 - Val Loss: 0.2682 - Val Accuracy: 0.9247
Validation Metrics - Precision: 0.9240 - Recall: 0.9239 - F1 Score: 

Bu eğitim döngüsü:
1. Her epoch içinde veriyi mini-batch'lere böler
2. Her mini-batch için:
   - **İleri Yayılım (Forward Propagation)**: Giriş verilerini ağ üzerinden ileterek tahminler üretir
   - **Kayıp Hesaplama**: Cross-entropy kayıp fonksiyonu ile tahmin hatasını ölçer
   - **Geri Yayılım (Backpropagation)**: Gradyanları hesaplayarak hatanın kaynağını bulur
   - **Ağırlık Güncelleme**: Gradyan inişi ile model parametrelerini günceller
3. Her epoch sonunda ortalama eğitim kaybını kaydeder

Bu kod bloğu her epoch sonunda:
1. Doğrulama veri seti üzerinde ileri yayılım gerçekleştirir
2. Doğrulama kaybını ve doğruluk oranını hesaplar
3. Precision, recall ve F1 skorlarını hesaplar
4. Tüm metrikleri ilgili listelere kaydeder ve ekrana yazdırır

Bu işlem, modelin eğitim verileri dışındaki performansını izlememizi sağlar.

## 7. Test Seti Değerlendirmesi

In [7]:
# Modelin Test Edilmesi
Z1_test = np.dot(x_test, W1) + b1
A1_test = relu(Z1_test)
Z2_test = np.dot(A1_test, W2) + b2
A2_test = softmax(Z2_test)

test_accuracy = np.mean(np.argmax(A2_test, axis=1) == np.argmax(y_test, axis=1))
test_precision, test_recall, test_f1 = calculate_metrics(y_test, A2_test)

print(f"\nTest Sonuçları:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1 Score: {test_f1:.4f}")

# Confuion Matrix hesaplama
y_true_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(A2_test, axis=1)

# Her sınıf için precision ve recall hesaplama (detaylı analiz)
class_precision = precision_score(y_true_class, y_pred_class, average=None)
class_recall = recall_score(y_true_class, y_pred_class, average=None)

print(f"\nSınıf Bazında Metrikler:")
for i in range(10):
    print(f"Sınıf {i} - Precision: {class_precision[i]:.4f} - Recall: {class_recall[i]:.4f}")


Test Sonuçları:
Accuracy: 0.9350
Precision: 0.9346
Recall: 0.9340
F1 Score: 0.9340

Sınıf Bazında Metrikler:
Sınıf 0 - Precision: 0.9450 - Recall: 0.9827
Sınıf 1 - Precision: 0.9712 - Recall: 0.9797
Sınıf 2 - Precision: 0.9317 - Recall: 0.9254
Sınıf 3 - Precision: 0.9069 - Recall: 0.9356
Sınıf 4 - Precision: 0.9168 - Recall: 0.9430
Sınıf 5 - Precision: 0.9399 - Recall: 0.8767
Sınıf 6 - Precision: 0.9237 - Recall: 0.9603
Sınıf 7 - Precision: 0.9538 - Recall: 0.9232
Sınıf 8 - Precision: 0.9286 - Recall: 0.9076
Sınıf 9 - Precision: 0.9289 - Recall: 0.9058


Eğitim tamamlandıktan sonra bu kod:
1. Test veri seti üzerinde modelin performansını değerlendirir
2. Genel doğruluk, precision, recall ve F1 skorlarını hesaplar
3. Her rakam sınıfı (0-9) için ayrı ayrı precision ve recall değerlerini hesaplar
4. Tüm sonuçları ekrana yazdırır

Bu kısım, modelin gerçek dünya verileri üzerindeki performansının en iyi göstergesidir.